## Find similar storm reports

This notebook shows a TFX pipeline that does a semantic search to find duplicate storm reports.
This is an example of a Workflow Pipeline that doesn't do any training. Instead, it just sets a TFX pipeline up for inference.

The source of our data are preliminary storm reports filed by storm spotters to National Weather Service offices. This dataset has already been manually cleaned, so for illustration, we'll ignore the year/location when doing the search

In [17]:
%pip install -q google-cloud-bigquery-storage pyarrow==0.16 tfx

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-io 0.11.0 requires tensorflow==2.1.0, but you'll have tensorflow 2.1.1 which is incompatible.
pandas-profiling 2.8.0 requires visions[type_image_path]==0.4.4, but you'll have visions 0.4.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
## CHANGE AS NEEDED
BEAM_RUNNER = 'DirectRunner'  # or DataflowRunner
PROJECT='ai-analytics-solutions'
BUCKET='ai-analytics-solutions-kfpdemo'
REGION='us-west1'

## Explore data in BigQuery

Preview data in BigQuery

In [2]:
%%bigquery
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  comments,
  'wind' as type,
FROM `bigquery-public-data.noaa_preliminary_severe_storms.wind_reports`
LIMIT 10

,year,julian_day,latitude,longitude,comments,type
0,2019,19,32.49,-85.18,TREES DOWN NEAR THE INTERSECTION OF LEE RD 440...,wind
1,2019,43,32.49,-85.13,REPORTS OF TREES DOWN IN VARIOUS LOCATIONS IN ...,wind
2,2019,62,32.60,-85.24,CORRECTS PREVIOUS TORNADO REPORT FROM SALEM. U...,wind
3,2019,85,32.55,-85.10,A TREE WAS DOWNED ONTO A HOME. (BMX),wind
4,2019,158,32.60,-85.42,TREE DOWN ON A HOME. TIME ESTIMATED FROM RADAR...,wind
5,2019,158,32.51,-85.42,MULTIPLE TREES DOWN ON LEE ROAD 29. TIME ESTIM...,wind
6,2019,158,32.71,-85.18,TREES DOWN IN BEULAH. TIME ESTIMATED FROM RADA...,wind
7,2019,158,32.54,-85.09,MULTIPLE TREES DOWN IN SMITHS STATION. TIME ES...,wind
8,2019,190,32.67,-85.49,TREES DOWN NEAR THE INTERSECTION OF LEE RD 147...,wind
9,2019,217,32.52,-85.07,CORRECTS PREVIOUS TSTM WND DMG REPORT FROM 1 E...,wind


In [3]:
%%bigquery
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  comments,
  size,
  'hail' as type
FROM `bigquery-public-data.noaa_preliminary_severe_storms.hail_reports`
LIMIT 10

,year,julian_day,latitude,longitude,comments,size,type
0,2019,177,0.00,-40.40,9134,100,hail
1,2019,222,0.00,-5.91,246 REPORTED AT 235 MEADOW VIEW DR... BUTTE MT...,175,hail
2,2019,84,32.77,-87.77,LOTS OF HAIL FELL. NOT QUITE THE SIZE OF A GOL...,125,hail
3,2019,84,32.77,-87.59,QUARTER SIZE HAIL... MAY HAVE BEEN LARGER. SEV...,100,hail
4,2019,73,33.66,-88.11,(BMX),150,hail
5,2019,136,33.89,-86.75,SEVERAL REPORTS OF DIME TO QUARTER SIZE HAIL I...,100,hail
6,2019,85,31.70,-87.80,TRAINED SPOTTER REPORTS QUARTER SIZED HAIL. RE...,100,hail
7,2019,74,34.29,-85.84,QUARTER SIZE HAIL WAS REPORTED IN THE COLLINSV...,100,hail
8,2019,84,32.73,-86.33,QUARTER-SIZED HAIL MEASURED IN THE TITUS AREA....,100,hail
9,2019,121,32.51,-86.21,QUARTER SIZE HAIL NEAR WALMART ON HIGHWAY 231....,100,hail


In [4]:
%%bigquery
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  LOWER(comments) AS comments,
  REGEXP_EXTRACT(comments, r"\([A-Z]+\)$") AS office,
  'tornado' as type
FROM `bigquery-public-data.noaa_preliminary_severe_storms.tornado_reports`
LIMIT 10

,year,julian_day,latitude,longitude,comments,office,type
0,2019,4,31.62,-85.28,tree damage from a brief tornado about 1.5 mi ...,(TAE),tornado
1,2019,19,32.49,-86.73,*** 2 inj *** corrects previous tornado report...,(BMX),tornado
2,2019,19,32.52,-86.24,*** 4 inj *** a nws storm survey team confirme...,(BMX),tornado
3,2019,19,32.85,-86.15,a nws storm survey team confirmed a tornado of...,(BMX),tornado
4,2019,54,33.06,-88.31,corrects previous tornado report from 3 wsw ga...,(BMX),tornado
5,2019,54,33.66,-88.06,the tornado touched down just south of crawfor...,(BMX),tornado
6,2019,54,33.70,-87.98,report of trees down along cody rd. near kingv...,(BMX),tornado
7,2019,55,33.83,-87.68,an ef-0 tornado touched down in a wooded area ...,(BMX),tornado
8,2019,55,32.86,-86.18,the tornado touched down just east of cr 40 an...,(BMX),tornado
9,2019,62,32.47,-86.75,a tornado touched 6 miles northwest of autauga...,(BMX),tornado


In [5]:
query = """
WITH wind AS (
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  comments,
  'wind' as type
FROM `bigquery-public-data.noaa_preliminary_severe_storms.wind_reports`
),

hail AS (
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  comments,
  'hail' as type
FROM `bigquery-public-data.noaa_preliminary_severe_storms.hail_reports`
),

tornadoes AS (
SELECT 
  EXTRACT(YEAR from timestamp) AS year,
  EXTRACT(DAYOFYEAR from timestamp) AS julian_day,
  latitude, longitude,
  comments,
  'tornado' as type
FROM `bigquery-public-data.noaa_preliminary_severe_storms.tornado_reports`
)

SELECT * FROM (
   SELECT * FROM wind
   UNION ALL
   SELECT * FROM hail
   UNION ALL
   SELECT * FROM tornadoes
)
"""

In [6]:
## skip_for_export
import google.cloud.bigquery as bq
df = bq.Client().query(query).result().to_dataframe()
df.groupby('type').count()

,year,julian_day,latitude,longitude,comments
type,,,,,
hail,5397,5397,5397,5397,5397
tornado,1677,1677,1677,1677,1677
wind,16064,16064,16064,16064,16064


## Ingest data

We'll use the TFX component BigQueryExampleGen to read in the data.

In [7]:
import tensorflow as tf
print('tensorflow ' + tf.__version__)
import tfx
print('tfx ' + tfx.__version__)
import apache_beam as beam
print('beam ' + beam.__version__)

tensorflow 2.1.1-dlenv_tfe
tfx 0.22.1
beam 2.23.0


In [8]:
from tfx.components import BigQueryExampleGen
example_gen = BigQueryExampleGen(query=query)

In [9]:
import os
beam_pipeline_args = [
    '--runner={}'.format(BEAM_RUNNER),
    '--project={}'.format(PROJECT),
    '--temp_location=' + os.path.join('gs://{}/noaa_similar_reports/'.format(BUCKET), 'tmp'),
    '--region=' + REGION,

    # Temporary overrides of defaults.
    '--disk_size_gb=50',
    '--experiments=shuffle_mode=auto',
    '--machine_type=n1-standard-8',
]

In [10]:
## skip_for_export
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [11]:
## skip_for_export
ingest_result = context.run(example_gen, beam_pipeline_args=beam_pipeline_args)

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1767: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


In [12]:
## skip_for_export
context.show(ingest_result)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/BigQueryExampleGen/examples/1, id: 1)]
        ))

In [13]:
## skip_for_export
print(ingest_result)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/BigQueryExampleGen/examples/1, id: 1)]
        ))


## Validate the data

Let's generate statistics from the data

In [14]:
from tfx.components import StatisticsGen
stats_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [15]:
## skip_for_export
context.run(stats_gen, beam_pipeline_args=beam_pipeline_args)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/StatisticsGen/statistics/2, id: 2)]
        ))

In [16]:
from tfx.components import SchemaGen
# This component only generates a new schema if one doesn't already exist.
schema_gen = SchemaGen(statistics=stats_gen.outputs['statistics'],
                       infer_feature_shape=True)

In [17]:
## skip_for_export
context.run(schema_gen, beam_pipeline_args=beam_pipeline_args)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/SchemaGen/schema/3, id: 3)]
        ))

In [18]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=stats_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

In [19]:
## skip_for_export
context.run(example_validator, beam_pipeline_args=beam_pipeline_args)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/ExampleValidator/anomalies/4, id: 4)]
        ))

## Preprocess data

Let's preprocess the comments:
* Pull out the office name from the comment. It's the last part of the comment (BMX)
* Lower case the comment

In [76]:
#%%writefile preprocess.py

import tensorflow_hub as hub
swivel = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")

def preprocessing_fn(inputs):
    import tensorflow as tf
    
    outputs = inputs.copy()
    comments = inputs['comments']
    outputs['office'] = tf.strings.substr(comments, -4, 3)
    comments = tf.strings.regex_replace(comments, r"\([A-Z]+\)$", "")
    outputs['comments'] = tf.strings.lower(comments)
    #if len(outputs['comments'].shape) == 0:
    #    swivel_input = [outputs['comments']]
    #else:
    #    swivel_input = outputs['comments']
    #outputs['embed'] = swivel(swivel_input)
    return outputs

In [77]:
## skip_for_export
inputs = df[:].iloc[3]
print(inputs['comments'])
outputs = preprocessing_fn(inputs)
print(outputs)
print(outputs['comments'])

A NWS STORM SURVEY TEAM CONFIRMED A TORNADO OF EF-1 INTENSITY... AFFECTING AREAS JUST SOUTHEAST OF ROCKFORD IN COOSA COUNTY ALONG COUNTY ROAD 101. (BMX)
year                                                       2019
julian_day                                                   19
latitude                                                  32.85
longitude                                                -86.15
comments      tf.Tensor(b'a nws storm survey team confirmed ...
type                                                    tornado
office                tf.Tensor(b'BMX', shape=(), dtype=string)
Name: 3, dtype: object
tf.Tensor(b'a nws storm survey team confirmed a tornado of ef-1 intensity... affecting areas just southeast of rockford in coosa county along county road 101. ', shape=(), dtype=string)


In [78]:
import tensorflow_transform as tft
from tfx.components import Transform
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('preprocess.py')
)

In [79]:
## skip_for_export
context.run(transform, beam_pipeline_args=beam_pipeline_args)

ExecutionResult(
    component_id: Transform
    execution_id: 12
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/Transform/transform_graph/12, id: 15)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-09-20T16_54_23.892298-by3sqlj6/Transform/transformed_examples/12, id: 16)]
        ))

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License